In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from math import sin, cos, asin, sqrt

In [2]:
df = pd.read_csv('project4.csv')
df.info()
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   flight_id            127 non-null    int64  
 1   flight_no            127 non-null    object 
 2   scheduled_departure  127 non-null    object 
 3   scheduled_arrival    127 non-null    object 
 4   departure_airport    127 non-null    object 
 5   arrival_airport      127 non-null    object 
 6   status               127 non-null    object 
 7   aircraft_code        127 non-null    object 
 8   actual_departure     127 non-null    object 
 9   actual_arrival       127 non-null    object 
 10  aircraft_code.1      127 non-null    object 
 11  aircraft_model       127 non-null    object 
 12  seat_count           127 non-null    int64  
 13  flight_id.1          118 non-null    float64
 14  bought_seats         118 non-null    float64
 15  sum_ticket           118 non-null    flo

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival,...,bought_seats,sum_ticket,arr_city,arr_lat,arr_lon,airport_code,dep_city,dep_lat,dep_lon,airport_code.1
77,136678,PG0480,2017-01-28T09:25:00Z,2017-01-28T10:15:00Z,AAQ,EGO,Arrived,SU9,2017-01-28T09:27:00Z,2017-01-28T10:17:00Z,...,80.0,631200.0,Belgorod,50.643799,36.590099,EGO,Anapa,45.002102,37.347301,AAQ
107,136887,PG0480,2017-01-20T09:25:00Z,2017-01-20T10:15:00Z,AAQ,EGO,Arrived,SU9,2017-01-20T09:30:00Z,2017-01-20T10:19:00Z,...,78.0,595200.0,Belgorod,50.643799,36.590099,EGO,Anapa,45.002102,37.347301,AAQ
114,136937,PG0480,2017-01-03T09:25:00Z,2017-01-03T10:15:00Z,AAQ,EGO,Arrived,SU9,2017-01-03T09:29:00Z,2017-01-03T10:19:00Z,...,81.0,626100.0,Belgorod,50.643799,36.590099,EGO,Anapa,45.002102,37.347301,AAQ
14,136185,PG0252,2017-01-14T10:05:00Z,2017-01-14T11:45:00Z,AAQ,SVO,Arrived,733,2017-01-14T10:06:00Z,2017-01-14T11:49:00Z,...,104.0,1520000.0,Moscow,55.972599,37.414600,SVO,Anapa,45.002102,37.347301,AAQ
61,136600,PG0480,2017-02-08T09:25:00Z,2017-02-08T10:15:00Z,AAQ,EGO,Arrived,SU9,2017-02-08T09:26:00Z,2017-02-08T10:16:00Z,...,97.0,765300.0,Belgorod,50.643799,36.590099,EGO,Anapa,45.002102,37.347301,AAQ


In [3]:
df.groupby(['arr_city'])['sum_ticket'].mean()
#в Новокузнецк рейсы летают пустыми, исключаем из рассмотрения

arr_city
Belgorod        7.104356e+05
Moscow          1.642576e+06
Novokuznetsk             NaN
Name: sum_ticket, dtype: float64

In [4]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 0 to 117
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   flight_id            118 non-null    int64  
 1   flight_no            118 non-null    object 
 2   scheduled_departure  118 non-null    object 
 3   scheduled_arrival    118 non-null    object 
 4   departure_airport    118 non-null    object 
 5   arrival_airport      118 non-null    object 
 6   status               118 non-null    object 
 7   aircraft_code        118 non-null    object 
 8   actual_departure     118 non-null    object 
 9   actual_arrival       118 non-null    object 
 10  aircraft_code.1      118 non-null    object 
 11  aircraft_model       118 non-null    object 
 12  seat_count           118 non-null    int64  
 13  flight_id.1          118 non-null    float64
 14  bought_seats         118 non-null    float64
 15  sum_ticket           118 non-null    flo

In [5]:
#подсчет времени полета
df['actual_arrival'] = pd.to_datetime(df['actual_arrival'], format='%Y-%m-%dT%H:%M:%SZ')
df['actual_departure'] = pd.to_datetime(df['actual_departure'], format='%Y-%m-%dT%H:%M:%SZ')
df['scheduled_departure'] = pd.to_datetime(df['scheduled_departure'], format='%Y-%m-%dT%H:%M:%SZ')
df['scheduled_arrival'] = pd.to_datetime(df['scheduled_arrival'], format='%Y-%m-%dT%H:%M:%SZ')

df['sch_flight_time'] = (df['scheduled_arrival']-df['scheduled_departure']).dt.total_seconds() / 60
df['act_flight_time'] = (df['actual_arrival']-df['actual_departure']).dt.total_seconds() / 60 

In [6]:
#подсчет расстояния полета
def airport_distance(lat1, lat2, lon1, lon2):
    R = 6371;  #Earth radius
    sin1 = sin((lat1 - lat2) / 2);
    sin2 = sin((lon1 - lon2) / 2);
    return 2 * R * asin(sqrt(sin1 * sin1 + sin2 * sin2 * cos(lat1) * cos(lat2))) / 1000

df['airport_distance'] = df.apply(lambda x: airport_distance(x['arr_lat'], x['dep_lat'],
                                                             x['arr_lon'], x['dep_lon']), axis=1)

In [7]:
#процент занятых мест
df['seat_perc'] = 100 / (df['seat_count'] / df['bought_seats'])

In [8]:
#удаление дубликатов столбцов, преобразуемых и не нужных для анализа столбцов
del_cols = ['airport_code.1','airport_code','flight_id.1','aircraft_code.1','actual_arrival','status',
            'actual_departure','scheduled_departure','scheduled_arrival','departure_airport','arrival_airport',
            'aircraft_code','dep_lat','arr_lat','dep_lon','arr_lon']
df = df.drop(columns = del_cols) 

In [9]:
#Исходя из найденной информации, удобнее считать расходы через время в полете, но можно использовать и расстояние.

In [10]:
#Расход топлива Boeing 737-300 составляет 2600 л/ч, Sukhoi Superjet-100 2300 л/ч.
#Средняя цена за литр авиационного топлива на 2017 год составляет 37.25 рублей.
#добавим эту информацию в датасет
def gas_price(model, time):
    if model == 'Boeing 737-300':
        cons = 2600 
    if model == 'Sukhoi Superjet-100':
        cons = 2300
    price = cons * 37.25 * time / 60
    return price

df['gas_price'] = df.apply(lambda x: gas_price(x['aircraft_model'], x['act_flight_time']), axis=1)

In [11]:
#Остальные расходы:
#Час работы командира самолета – 2780 р, второго пилота - 1670 р, 4 бортпроводника по 1000 р 
#=> экипаж получает 8450 р/час.
#Комиссия аэропорта – 288 р за тонну максимальной взлетной массы + 225 р за тонну на обеспечение авиационной 
#безопасности. Вес Boeing 737-300 - 62 т, Sukhoi Superjet-100 - 46 т.
#302 р на обслуживание каждого пассажира
#ТО самолета - 23 100 р на час полета
#Неполетные расходы - 15% цены билета

In [12]:
def other_costs(model, time, pas, sum_ticket):
    if model == 'Boeing 737-300':
        w = 62 
    if model == 'Sukhoi Superjet-100':
        w = 46
    sum = (288 + 225) * w + 302 * pas + 0.15 * sum_ticket + (8450 + 23100) * time / 60 
    return sum

df['other'] = df.apply(lambda x: other_costs(x['aircraft_model'], x['act_flight_time'],
                                             x['bought_seats'], x['sum_ticket']), axis=1)

In [13]:
df['total'] = df['sum_ticket'] - (df['other'] + df['gas_price'])
df['total'].describe()

#убыточных рейсов не обнаружено

count    1.180000e+02
mean     7.858838e+05
std      3.431166e+05
min      3.095285e+05
25%      4.843135e+05
50%      7.070866e+05
75%      1.104860e+06
max      1.318034e+06
Name: total, dtype: float64

In [14]:
df['total_perc'] = 100 / (df['sum_ticket'] / (df['other'] + df['gas_price']))
df['total_perc'].describe()

count    118.000000
mean      34.105879
std        2.520704
min       30.114846
25%       32.077457
50%       34.481291
75%       35.133641
max       41.708380
Name: total_perc, dtype: float64

In [15]:
#не учитывая конкретные цены, согласно другому источнику:
#Расходы на топливо составляют  12.7% от всех издержек авиакомпании.
df['other_p'] = df['gas_price'] * 0.873 / 0.127
df['total_p'] = df['sum_ticket'] - (df['other_p'] + df['gas_price'])
df['total_p'].describe()

count       118.000000
mean     260521.986076
std      155803.114161
min      -31171.916010
25%      173816.633858
50%      203128.083990
75%      366797.637795
max      615002.624672
Name: total_p, dtype: float64

In [16]:
df['total_perc_p'] = 100 / (df['sum_ticket'] / (df['other_p'] + df['gas_price']))
df['total_perc_p'].describe()

count    118.000000
mean      78.880385
std        7.470037
min       67.391165
25%       74.067446
50%       76.491861
75%       82.385698
max      105.870417
Name: total_perc_p, dtype: float64

In [17]:
df[df['total_p']<0]

#обнаружено 2 убыточных рейса

,flight_id,flight_no,aircraft_model,seat_count,bought_seats,sum_ticket,arr_city,dep_city,sch_flight_time,act_flight_time,airport_distance,seat_perc,gas_price,other,total,total_perc,other_p,total_p,total_perc_p
68,136642,PG0480,Sukhoi Superjet-100,97,64.0,531000.0,Belgorod,Anapa,50.0,49.0,5.344716,65.979381,69967.916667,148341.833333,312690.25,41.112947,480960.561024,-19928.47769,103.753009
93,136807,PG0480,Sukhoi Superjet-100,97,68.0,531000.0,Belgorod,Anapa,50.0,50.0,5.344716,70.103093,71395.833333,150075.666667,309528.50,41.708380,490776.082677,-31171.91601,105.870417
